In [9]:
#https://github.com/YatingMusic/miditoolkit/blob/master/examples/parse_and_create_MIDI_files.ipynb

import os
from miditoolkit.midi import parser as mid_parser
from tqdm import tqdm
import pandas as pd


def split_scores_into_chunks(scores_df,paths_column_name,n_chunks,outputs_paths_column_name):

    full_chunks_paths_list=[]
    
    paths=scores_df[paths_column_name]
    paths=paths.apply(lambda x: x.replace('../../author-profiling-in-symbolic-music/','../'))    
    
    for score_path in tqdm(paths, desc='generating chunk files'):

        mido_obj = mid_parser.MidiFile(score_path)

        chunk_duration=mido_obj.max_tick//n_chunks

        output_path=score_path.replace('/merged piano scores/','/merged piano scores chunks/').split('/')
        output_path='/'.join(output_path)
        output_dir = os.path.dirname(output_path)
        os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists
        
        chunks_paths_list=[]

        for chunk in range(n_chunks):

            full_output_path=f'../{output_path}chunk{chunk}.midi'
            chunks_paths_list.append(full_output_path)
            #mido_obj.dump(full_output_path, segment=((chunk)*chunk_duration, (chunk+1)*chunk_duration))     


        full_chunks_paths_list.append(chunks_paths_list)
            
    print('generating dataframe with new list')

    scores_df[outputs_paths_column_name]=full_chunks_paths_list
    
    return scores_df

### 1. Split type0 scores

In [ ]:
type0_scores=pd.read_csv('../dataframes/type0_scores.csv')

In [ ]:
type0_chunks_df=split_scores_into_chunks(type0_scores,'type0_path',3)

In [ ]:
type0_chunks_df=type0_chunks_df.explode('type0_chunks_paths').reset_index(drop=True)
type0_chunks_df.to_csv('./dataframes/type0_chunks_scores.csv')

### 2. Split piano merged type 0 scores

In [7]:
piano_merged_type0_df=pd.read_csv('../dataframes/piano_merged_scores.csv')

In [10]:
piano_merged_type0_chunks_df=split_scores_into_chunks(scores_df=piano_merged_type0_df,
                                                      paths_column_name='paths_type0_piano',
                                                       n_chunks=2,
                                                      outputs_paths_column_name='piano_merged_type0_chunks_paths')

generating chunk files: 100%|██████████| 1354/1354 [00:11<00:00, 117.41it/s]

generating dataframe with new list


In [15]:
piano_merged_type0_chunks_df=piano_merged_type0_chunks_df.explode('piano_merged_type0_chunks_paths').reset_index(drop=True)
piano_merged_type0_chunks_df.to_csv('../dataframes/piano_merged_type0_chunks_scores.csv',index=False)